In [1]:
import numpy as np 
import math
from scipy.integrate import dblquad, quad, nquad
from IPython.display import display, Math
from antennaOG import AntennaModel
import sympy as smp
import scipy as sp
from sympy import symbols, integrate, sec



In [2]:
capacity_link = r"C B = \frac{1}{2} \log \left(1 + \frac{PB}{N0}\right)"
psec = r"p sec = \frac{1}{As} \iint_S C \sec(x, y) \, dx \, dy = \frac{F \sec}{As}"

CAPACITY LINK EQUATION:

In [3]:
display(Math(capacity_link))

<IPython.core.display.Math object>

In [4]:
def capacity_link(prx):
        cp = 1/2*np.log(1+(prx/(-90+0)))
        return cp

PRESSURE SECRECY EQUATION:

In [5]:
display(Math(psec))

<IPython.core.display.Math object>

In [6]:
def secpres(prx, pe, As):
    f = lambda x,y :(1/2)*np.log(1+(prx/(-30)))-(1/2)*np.log(1+(pe/(-30)))
    integral, error = dblquad(f,0, 15, lambda x: 0, lambda x: 15)
    return integral

In [7]:
def eavesdroppergain(x,y):
    AgE = 8 - np.min(-np.min([12*((np.arctan(y/x))-90/65)**2,30]))+(-np.min([12*((np.arctan(y/x))/65)**2,30]))
    return AgE

def transmittergain():
    AgT = 8 - np.min(-np.min([12*((np.arctan(0))-90/65)**2,30]))+(-np.min([12*((np.arctan(0))/65)**2,30]))
    return AgT

In [36]:
prx = 21.43562
csec = lambda x,y : 1/2*np.log(1+(prx/(-30))) - 1/2*np.log(1+((eavesdroppergain(x,y))/(-30))) 

def integral(function):
    integral, error = dblquad(func = csec,a = 5, b = 10, gfun=5, hfun=10)
    return integral, error

print(integral(csec))



(-8.829119474622619, 9.802291727892397e-14)


In [ ]:
1/2*np.log(1+(prx/(-30)))

0.26956677344849395

In [10]:
int(math.degrees(np.arctan(14/6)))

66

In [59]:
def calc_path_loss_los(fc_ghz, d_2d_meter):
        H_UT  = 1.5 # Height of UT (m).
        H_BS  = 10 # Height of BS (m).
        d_bp = 4*H_BS*H_UT*fc_ghz/0.3 # Breaking point distance.
        d_3d = np.sqrt(np.power(d_2d_meter, 2) + np.power(H_BS-H_UT, 2))
        pl = np.empty_like(d_2d_meter).astype(np.float64)
        # PL1: d_2d <= d_bp
        cond = d_2d_meter <= d_bp
        # pl[cond] = 28 + 22*np.log10(d_3d[cond]) + 20*np.log10(fc_ghz)
        pl[cond] = 32.4 + 21*np.log10(d_3d[cond]) + 20*np.log10(fc_ghz)
        # PL2: d_2d > d_bp
        cond = np.invert(cond)
        # pl[cond] = 28 + 40*np.log10(d_3d[cond]) + 20*np.log10(fc_ghz) \
        #     - 9*np.log10(np.power(d_bp, 2)+np.power(H_BS-H_UT, 2))
        pl[cond] = 32.4 + 40*np.log10(d_3d[cond]) + 20*np.log10(fc_ghz) \
            - 9.5*np.log10(np.power(d_bp, 2)+np.power(H_BS-H_UT, 2))
        return pl


def get_dis(loc):
        return np.sqrt(loc[0]**2 + loc[1]**2)


In [86]:
test = AntennaModel(30)

def rseve(x,y):
    return 25 + test.calc_ant_gain(90, np.arctan(y/x),31,31) - calc_path_loss_los(25, get_dis([x,y]))

prx = -21.43562
csec = lambda x,y : 1/2*np.log(1+(prx/(-30))) - 1/2*np.log(1+rseve(x,y)/(-30))

def integral(function):
    integral, error = dblquad(func = csec,a = 5, b = 10, gfun=0, hfun=5)
    return integral, error


print(integral(csec))


/home/adrian/.local/lib/python3.8/site-packages/scipy/integrate/quadpack.py:864: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


(-9.78849935726318, 1.052516971355999e-07)


In [66]:
1/2*np.log(1+rseve(1,5)/(-30))

0.6355738185601486

In [85]:
# 1/2*np.log() 

ue_loc = [5,0]
print(1/2*np.log(1+(prx/(-30)))-((1/2*np.log(1+(25+(test.calc_ant_gain(90,25,20,20))-calc_path_loss_los(25,get_dis(ue_loc)))/-30))))

-0.32437451498948994


In [61]:
1+(25+(test.calc_ant_gain(90,25,31,31))-calc_path_loss_los(25,get_dis(ue_loc)))

-55.577980823330066

In [56]:
1+(test.calc_ant_gain(90,-12,31,31))

-15.688524544741611

In [13]:
test.calc_ant_gain(0,0,45,45)

-115.00591715976331

In [14]:
np.arctan(10/8)

0.8960553845713439

In [52]:
def f(x,y):
    return (1/2*np.log(max(0,1+(test.calc_ant_gain(90,np.degrees(np.arctan(y/x)),31,31)))))

dblquad(f, 0, 4, lambda x: 0, lambda x: 4)

/tmp/ipykernel_69592/817448270.py:2: RuntimeWarning: divide by zero encountered in log
  return (1/2*np.log(max(0,1+(test.calc_ant_gain(90,np.degrees(np.arctan(y/x)),31,31)))))


(-inf, inf)

In [27]:
answer = integral(csec)

/home/adrian/.local/lib/python3.8/site-packages/scipy/integrate/quadpack.py:864: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


KeyboardInterrupt: 

In [ ]:
print(answer)

(-8.72659510344771, 9.688466810428798e-14)


In [ ]:
nine = secpres(25.67853, 19.2382, 9)
twentyfive = secpres(25.67853,19.2382, 25)
exhaustive = secpres(25.67853,19.2382, 63)
print("pressure if sweep space is 9 beam:" ,nine)
print("pressure if sweep space is 25 beam:" ,twentyfive)
print("pressure if sweep space is 63 beam:" ,exhaustive)

pressure if sweep space is 9 beam: -102.64581054402338
pressure if sweep space is 25 beam: -102.64581054402338
pressure if sweep space is 63 beam: -102.64581054402338


In [ ]:
def sum (x,y): 
    return x+y 

lambda x,y: x+y

<function __main__.<lambda>(x, y)>

In [ ]:
pe = 26.06179973983887
prx = 25.69825527778019

In [ ]:
def arb_calc(x,y):
    return prx - (pe*x)+(pe*y)

In [ ]:
(1/2)*np.log(1+(prx/(-30)))-(1/2)*np.log(1+(pe/(-30)))


0.0441484287845485

In [ ]:
#the following is an integration of x times y
# g = lambda x,y: arb_calc(x,y)
intergal, error = dblquad(arb_calc, 0, 15, 0, 15)
secpres = intergal/225
print(secpres)

25.69825527778019


TESTING: